Nathaniel's Branch

In [2]:
# Initalize GPU
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os 
if ! ( os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv") ):
  %cd /content/drive/MyDrive/CAIS_Sentiment140_Project/
  !unzip trainingandtestdata.zip # type: ignore
  %ls

/content/drive/MyDrive/CAIS_Sentiment140_Project
Archive:  trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
savedtensors/                   training.1600000.processed.noemoticon.csv
testdata.manual.2009.06.14.csv  trainingandtestdata.zip


In [5]:
import pandas as pd
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text' ])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 1,600,000



In [6]:
# Display 10 random rows from the data.
df.sample(10)

,target,ids,date,flag,user,text
1023189,4,1882953754,Fri May 22 07:46:01 PDT 2009,NO_QUERY,pulaaa16,@jonasbrothers http://twitpic.com/5llux - i wa...
136465,0,1880101624,Fri May 22 00:06:13 PDT 2009,NO_QUERY,paulney,Can't believe #YEG made the front page of Digg...
8650,0,1548304302,Fri Apr 17 20:35:14 PDT 2009,NO_QUERY,nettefrawl,@ihenpecked leakage huh? I'm afraid I missed s...
1496282,4,2070090888,Sun Jun 07 17:04:45 PDT 2009,NO_QUERY,threesunset,@jakks well I got your back in Boston #noonef...
24813,0,1558130716,Sun Apr 19 07:22:16 PDT 2009,NO_QUERY,_sars_,"Damn, i thought Yamadonga was on TV 2day! Cant..."
51953,0,1679209895,Sat May 02 08:48:16 PDT 2009,NO_QUERY,artisaweapon,@stormygyrltx it is a bit soupy out
348292,0,2016724003,Wed Jun 03 07:15:25 PDT 2009,NO_QUERY,merrymorgantown,@robynnnnnn be ashamed! just at the eurostar b...
387330,0,2053990153,Sat Jun 06 06:16:11 PDT 2009,NO_QUERY,NicolyTamy,@MelissaH27 I don't know I really miss my ork...
1174364,4,1980941774,Sun May 31 08:02:53 PDT 2009,NO_QUERY,iAmSongByrd,Gettin' Dressed for Church!! &amp; I Jus Notic...
1580392,4,2190058651,Tue Jun 16 01:39:01 PDT 2009,NO_QUERY,GeorginaMeehan,"Had such gd time @jonasbrothers gig,nw ave no ..."


In [7]:
print("0, negative sentiment")
df.loc[df.target == 0].sample(5)[['target', 'text']]

0, negative sentiment


,target,text
775799,0,@livelovenonstop where are you going on the 19...
273007,0,@brainscrewer Me too. I also hope that my int...
456978,0,wondering why nobody wants to follow me??? Crap!
365353,0,"Is using the Google Street View. Man, I can se..."
374141,0,woke up sometime back. Realised her phone scre...


In [8]:
print("4, positive sentiment")
df.loc[df.target == 4].sample(5)[['target', 'text']]

4, positive sentiment


,target,text
1568418,4,is going to beddd good nighttt.
1003697,4,@samuelcoates Am I missing some kind of joke ...
877527,4,@Zorlone Do they come with web hosting too?
965188,4,getting ready.
1254927,4,@wittyaI could bring you mine on break lol


In [9]:
#Extract text and target labels
texts = df.text.values
targets = df.target.values

In [10]:
!pip install transformers
import transformers
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Loading BERT tokenizer...


#Potential To-Do = Further Scrub Data of mentions, links, and other punctuation

In [11]:
# Print the original sentence.
print(' Original: ', texts[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(texts[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

 Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Tokenized:  ['@', 'switch', '##foot', 'http', ':', '/', '/', 't', '##wi', '##tp', '##ic', '.', 'com', '/', '2', '##y', '##1', '##z', '##l', '-', 'aw', '##w', '##w', ',', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', ';', 'd']
Token IDs:  [1030, 6942, 13064, 8299, 1024, 1013, 1013, 1056, 9148, 25856, 2594, 1012, 4012, 1013, 1016, 2100, 2487, 2480, 2140, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040]


In [12]:
if os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv"):
  input_ids = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/input_ids.pt")
  attention_masks = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/attention_masks.pt")
  targets = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/targets.pt")
else:
  input_ids = []
  attention_masks = []

  # For every sentence...
  for t in texts:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          t,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 140,           # Pad & truncate all sentences.
                          pad_to_max_length = True,          # Pad & truncate all sentences.
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt'     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  targets = torch.tensor(targets)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', texts[0])
  print('Token IDs:', input_ids[0])

In [13]:
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])
print('Attention Masks', attention_masks[0])
print("Targets", targets[0])

Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Token IDs: tensor([  101,  1030,  6942, 13064,  8299,  1024,  1013,  1013,  1056,  9148,
        25856,  2594,  1012,  4012,  1013,  1016,  2100,  2487,  2480,  2140,
         1011, 22091,  2860,  2860,  1010,  2008,  1005,  1055,  1037, 26352,
         5017,  1012,  2017,  2323,  2050,  2288,  2585, 12385,  1997,  2353,
         2154,  2000,  2079,  2009,  1012,  1025,  1040,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
    

In [14]:
if os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv"):
  print("Hey, It's Joey here.")
else:
  newpath = r'/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors' 
  if not os.path.exists(newpath):
      os.makedirs(newpath)

  torch.save(texts, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/texts.pt" )
  torch.save(input_ids, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/input_ids.pt" )
  torch.save(attention_masks, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/attention_masks.pt" )
  torch.save(targets, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/targets.pt" )

Hey, It's Joey here.


#Word Embedding Step Done

In [15]:
#Test Train Val Split

from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, targets)
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


1,520,000 training samples
80,000 validation samples


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order (RandomSampler)
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially (SequentialSampler)
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top; ensure the model does NOT outputs attentions and hidden_states 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps = 1e-8 )

In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = epochs * len(train_dataloader)

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [20]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [22]:
checkpoint = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pth")
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_i = checkpoint['epoch']

print(checkpoint['model_state_dict'])
print(checkpoint['optimizer_state_dict'])
print(checkpoint['epoch'])



FileNotFoundError: ignored

In [ ]:
import random
import numpy as np

seed_val = 28
load_from_checkpoint = False

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


if load_from_checkpoint:
  checkpoint = torch.load('/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pth')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch_i = checkpoint['epoch']
  step = checkpoint['step']
  batch = checkpoint['batch']
  loss = checkpoint['loss']


# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            torch.save({'model_state_dict': model.state_dict(),
                        'epoch': epoch_i,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'step':step,
                        'batch':batch,
                        'loss':total_train_loss},
                        "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pth"
                        )

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)

        # 0 the gradients
        model.zero_grad() 

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.       
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask= b_input_mask, 
                       labels= b_labels,
                       return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end
        total_train_loss += loss.item()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].float().to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

torch.save(model.state_dict(), "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/savedfinalmodel.pth")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  47,500.    Elapsed: 0:00:30.
  Batch    80  of  47,500.    Elapsed: 0:01:03.
  Batch   120  of  47,500.    Elapsed: 0:01:38.
  Batch   160  of  47,500.    Elapsed: 0:02:13.
  Batch   200  of  47,500.    Elapsed: 0:02:48.
  Batch   240  of  47,500.    Elapsed: 0:03:23.
  Batch   280  of  47,500.    Elapsed: 0:03:58.
  Batch   320  of  47,500.    Elapsed: 0:04:33.
  Batch   360  of  47,500.    Elapsed: 0:05:08.
  Batch   400  of  47,500.    Elapsed: 0:05:43.
  Batch   440  of  47,500.    Elapsed: 0:06:18.
  Batch   480  of  47,500.    Elapsed: 0:06:52.
  Batch   520  of  47,500.    Elapsed: 0:07:28.
  Batch   560  of  47,500.    Elapsed: 0:08:02.
  Batch   600  of  47,500.    Elapsed: 0:08:37.
  Batch   640  of  47,500.    Elapsed: 0:09:12.
  Batch   680  of  47,500.    Elapsed: 0:09:47.
  Batch   720  of  47,500.    Elapsed: 0:10:22.
  Batch   760  of  47,500.    Elapsed: 0:10:57.
  Batch   800  of  47,500.    Elapsed: 0:11:3